### Este notebook contém os comandos para estruturar o esquema bronze.
1. Importar as bibliotecas. 

In [0]:
# !!! ATENÇÃO !!!
# Caso não consiga ler o arquivo excel, pode ser necessário instalar a biblioteca 'xlrd'
#%pip install xlrd>=2.0.1

In [0]:
# !!! ATENÇÃO !!!
# Caso instale alguma biblioteca, será necessário reiniciar o kernel
#dbutils.library.restartPython()

In [0]:
# Importando bibliotecas

import pandas as pd

2. Criar a tabela *beneficiarios_bpc*.

In [0]:
# Cria a tabela 'beneficiarios_bpc' no schema 'bronze' do catálogo 'bpc'

spark.sql("USE CATALOG bpc")  # Seleciona o catálogo 'bpc'
spark.sql("USE SCHEMA bronze")  # Seleciona o schema 'bronze'

# Lê o arquivo CSV dos beneficiários do BPC, usando cabeçalho e separador vírgula
df_ben_bpc = spark.read.option("header", True) \
    .option("sep", ",") \
    .csv("dbfs:/Volumes/bpc/staging/imdb/Beneficiarios_BPC_2025-10.csv")

display(df_ben_bpc.limit(3))  # Exibe as 3 primeiras linhas do DataFrame para conferência

# Salva o DataFrame como uma tabela Delta chamada 'beneficiarios_bpc', sobrescrevendo se já existir
df_ben_bpc.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("beneficiarios_bpc")

# Adiciona um comentário descritivo à tabela criada
spark.sql("""
  COMMENT ON TABLE bpc.bronze.beneficiarios_bpc IS 
  'Tabela de beneficiários do BPC, contendo dados por município, mês, tipo de benefício e valores.'
""")  

# Adiciona um comentário descritivo a coluna 'ibge'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.IBGE IS 'Código IBGE do município ao qual o dado se refere.'
""")  

# Adiciona um comentário descritivo a coluna 'anomes'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.ANOMES IS 'Ano e mês de referência do dado.'
""")  

# Adiciona um comentário descritivo a coluna 'bpc_ben'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_BEN IS 'Quantidade **total** de beneficiários do BPC.'
""")  

# Adiciona um comentário descritivo a coluna 'bpc_pcd_ben'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_PCD_BEN IS 'Quantidade de beneficiários do BPC na categoria **Pessoa com Deficiência (PcD)**.'
""")  

# Adiciona um comentário descritivo a coluna 'bpc_idoso_ben'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_IDOSO_BEN IS 'Quantidade de beneficiários do BPC na categoria **Idoso**.'
""")

# Adiciona um comentário descritivo a coluna 'bpc_pcd_val'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_PCD_VAL IS '**Valor total** pago para beneficiários PcD do BPC no período.'
""")  

# Adiciona um comentário descritivo a coluna 'bpc_idoso_val'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_IDOSO_VAL IS '**Valor total** pago para beneficiários Idosos do BPC no período.'
""")  

# Adiciona um comentário descritivo a coluna 'bpc_val'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.beneficiarios_bpc.BPC_VAL IS '**Valor total** pago de BPC para todas as categorias.'
""")  

display("Tabela criada com sucesso.")

ibge,anomes,bpc_ben,bpc_pcd_ben,bpc_idoso_ben,bpc_pcd_val,bpc_idoso_val,bpc_val
110001,202510,690,581,109,881976.58,165462,1047438.58
110002,202510,4122,2839,1283,4315847.99,1949137.91,6264985.90
110003,202510,105,86,19,130553.28,28842,159395.28


'Tabela criada com sucesso.'

* Tabela *beneficiarios_bpc* criada:

![Beneficiarios bpc](imagens/03-Esquema_bronze-Tabela_beneficiarios_bpc.png)

3. Criar a tabela *municipios_ibge*.

In [0]:
# Cria a tabela 'municipios_ibge' no schema 'bronze' do catálogo 'bpc'

spark.sql("USE CATALOG bpc")  # Seleciona o catálogo 'bpc'
spark.sql("USE SCHEMA bronze")  # Seleciona o schema 'bronze'

# Lê o arquivo xls dos municípios do IBGE
df_excel = pd.read_excel("/Volumes/bpc/staging/imdb/Mun_Faixa_de_Fronteira_Cidades_Gemeas_2024.xls")

# !!! ATENÇÃO !!!
# Remove caracteres inválidos dos nomes das colunas
df_excel.columns = [
    col.replace(" ", "_")
    .replace(";", "_")
    .replace("{", "_")
    .replace("}", "_")
    .replace("(", "_")
    .replace(")", "_")
    .replace("\n", "_")
    .replace("\t", "_")
    .replace("=", "_")
    for col in df_excel.columns
]

# Converte o DataFrame para CSV
df_excel.to_csv("/Volumes/bpc/staging/imdb/Mun_Faixa_de_Fronteira_Cidades_Gemeas_2024.csv", index=False)

# Lê o arquivo csv dos municípios do IBGE 
df_mun_ibge = spark.read.option("header", True) \
    .option("sep", ",") \
    .csv("dbfs:/Volumes/bpc/staging/imdb/Mun_Faixa_de_Fronteira_Cidades_Gemeas_2024.csv")

display(df_mun_ibge.limit(3))  # Exibe as 3 primeiras linhas do DataFrame para conferência

# Salva o DataFrame como uma tabela Delta chamada 'municipios_ibge', sobrescrevendo se já existir
df_mun_ibge.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("municipios_ibge")

# Adiciona um comentário descritivo à tabela criada
spark.sql("""
  COMMENT ON TABLE bpc.bronze.municipios_ibge IS 
  'Tabela dos municípios do IBGE, contendo dados por município, região, estado, etc.'
""")  

# Adiciona um comentário descritivo a coluna 'cd_mun'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CD_MUN IS 'Código do Município (IBGE).'
""")  

# Adiciona um comentário descritivo a coluna 'nm_mun'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.NM_MUN IS 'Nome do município.'
""")  

# Adiciona um comentário descritivo a coluna 'cd_rgi'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CD_RGI IS 'Código da Região Imediata (Antiga Microrregião)'
""")

# Adiciona um comentário descritivo a coluna 'nm_rgi'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.NM_RGI IS 'Nome da Região Imediata)'
""")

# Adiciona um comentário descritivo a coluna 'cd_rgint'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CD_RGINT IS 'Código da Região Intermediária (Antiga Mesorregião)'
""")

# Adiciona um comentário descritivo a coluna 'nm_rgint'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.NM_RGINT IS 'Nome da Região Intermediária'
""")

# Adiciona um comentário descritivo a coluna 'cd_uf'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CD_UF IS 'Código da Unidade da Federação'
""")

# Adiciona um comentário descritivo a coluna 'nm_uf'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.NM_UF IS 'Nome da Unidade da Federação'
""")

# Adiciona um comentário descritivo a coluna 'sigla_uf'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.SIGLA_UF IS 'Sigla da Unidade da Federação'
""")

# Adiciona um comentário descritivo a coluna 'cd_regiao'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CD_REGIAO IS 'Código da Região (Grande Região)'
""")

# Adiciona um comentário descritivo a coluna 'nm_regiao'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.NM_REGIAO IS 'Nome da Região (Grande Região)'
""")

# Adiciona um comentário descritivo a coluna 'sigla_rg'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.SIGLA_RG IS 'Sigla da Região'
""")

# Adiciona um comentário descritivo a coluna 'area_tot'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.AREA_TOT IS 'Área total do município em km²'
""")

# Adiciona um comentário descritivo a coluna 'toca_lim'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.TOCA_LIM IS 'Indica se o limite municipal toca o limite internacional.'
""")

# Adiciona um comentário descritivo a coluna 'area_int'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.AREA_INT IS 'Área em km² do município que se encontra dentro da Faixa de Fronteira.'
""")

# Adiciona um comentário descritivo a coluna 'porc_int'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.PORC_INT IS 'Porcentagem da área total do município que se encontra dentro da Faixa de Fronteira.'
""")

# Adiciona um comentário descritivo a coluna 'faixa_sede'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.FAIXA_SEDE IS 'Indica se a sede municipal se encontra dentro da Faixa de Fronteira.'
""")

# Adiciona um comentário descritivo a coluna 'cid_gemea'
spark.sql("""
  COMMENT ON COLUMN bpc.bronze.municipios_ibge.CID_GEMEA IS 'Indica se a sede municipal é uma Cidade Gêmea. Caso preenchido, sim.'
""")

display("Tabela criada com sucesso.")


CD_MUN,NM_MUN,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_UF,NM_UF,SIGLA_UF,CD_REGIAO,NM_REGIAO,SIGLA_RG,AREA_TOT,TOCA_LIM,AREA_INT,PORC_INT,FAIXA_SEDE,CID_GEMEA
1100015,Alta Floresta D'Oeste,110005.0,Cacoal,1102.0,Ji-Paraná,11,Rondônia,RO,1,Norte,N,7067.127,SIM,7067.127,100.0,sim,null
1100031,Cabixi,110006.0,Vilhena,1102.0,Ji-Paraná,11,Rondônia,RO,1,Norte,N,1314.352,SIM,1314.352,100.0,sim,null
1100056,Cerejeiras,110006.0,Vilhena,1102.0,Ji-Paraná,11,Rondônia,RO,1,Norte,N,2783.297,NÃO,2783.297,100.0,sim,null


'Tabela criada com sucesso.'

* Tabela *municipios_ibge* criada:

![Municípios ibge](imagens/03-Esquema_bronze-Tabela_municipios_ibge.png)

4. Esquema **bronze** criado.